# 03 — Data Preparation (CRISP-DM)

Nesta etapa do CRISP-DM, os dados brutos são preparados para análise e
modelagem, incluindo limpeza, padronização, tratamento de valores ausentes
e anonimização de informações sensíveis.

O objetivo é gerar uma base consistente, confiável e reutilizável para
as próximas fases do projeto.

In [63]:
import re
import hashlib
import unicodedata
import numpy as np
import pandas as pd
from pathlib import Path

In [21]:
DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df.shape

(24009, 16)

In [22]:
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.replace(r"[^a-z0-9_]", "_", regex=True)
    )

    return df

df.columns

Index(['Referência', 'Nome', 'Cargo', 'Data Admissão', 'Tipo de Regime',
       'Descontos', 'Liquido', 'Data Desligamento', 'Proventos', 'Contrato',
       'Atividade', 'Nome Atividade', 'Tipo de Contrato',
       'Data Prevista Termino Contrato', 'Carga Horária (Sem.)',
       'arquivo_origem'],
      dtype='object')

## Renomeando colunas

In [23]:
# Criando df_prepared como cópia de df_raw
df_prepared = df.copy()

# renomeando colunas
df_prepared.rename(columns={
    "Referência": "referencia",
    "Nome": "nome",
    "Cargo": "cargo",
    "Data Admissão": "data_admissao",
    "Tipo de Regime": "tipo_regime",
    "Descontos": "descontos",
    "Liquido": "liquido",
    "Data Desligamento": "data_desligamento",
    "Proventos": "proventos",
    "Contrato": "contrato",
    "Atividade": "atividade",
    "Nome Atividade": "nome_atividade",
    "Tipo de Contrato": "tipo_contrato",
    "Data Prevista Termino Contrato": "data_prevista_termino_contrato",
    "Carga Horária (Sem.)": "carga_horaria_semanal"
}, inplace=True)

In [24]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv


## Removendo as linhas de somas finais que vem em cada dataset

In [25]:
df_prepared = df_prepared.dropna(subset=["referencia"])

In [26]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv


In [27]:
# Extrair tipo de pagamento e mês diretamente da coluna referencia
df_prepared[["tipo_pagamento_raw", "mes_referencia"]] = df_prepared["referencia"].str.split(" - ", expand=True)

# Padronizar tipo de pagamento
df_prepared["tipo_pagamento"] = df_prepared["tipo_pagamento_raw"].replace({
    "Folha Mensal": "folha_mensal",
    "Folha Complementar": "vale_alimentacao"
})

# remover coluna temporária
df_prepared = df_prepared.drop(columns=["tipo_pagamento_raw"])

# conferir resultados
df_prepared[["referencia", "tipo_pagamento", "mes_referencia"]].head()

,referencia,tipo_pagamento,mes_referencia
0,Folha Complementar - Abril,vale_alimentacao,Abril
1,Folha Mensal - Abril,folha_mensal,Abril
2,Folha Complementar - Abril,vale_alimentacao,Abril
3,Folha Mensal - Abril,folha_mensal,Abril
4,Folha Complementar - Abril,vale_alimentacao,Abril


In [28]:
df_prepared.tail()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento
24003,Folha Mensal - Setembro,YASMIN MICKAELA TAMINI,AGENTE DE DESENVOLVIMENTO INFANTIL,17/01/2025,REGIME PROPRIO,"535,56","2.920,63",NaN,"3.456,19",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv,Setembro,folha_mensal
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv,Setembro,vale_alimentacao
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv,Setembro,folha_mensal
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv,Setembro,folha_mensal
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv,Setembro,vale_alimentacao


## Inferência manual do sexo dos servidores — Processo

### Extração dos nomes
- A partir do dataset preparado, foram extraídos os nomes dos servidores.
- Os nomes foram deduplicados para garantir que cada servidor fosse analisado apenas uma vez.

### Normalização dos nomes
- Os nomes foram padronizados para facilitar comparações posteriores.
- A normalização consistiu em:
  - Conversão para letras maiúsculas
  - Remoção de espaços em branco no início e no fim do texto

### Criação de DataFrame auxiliar
- Foi criado um DataFrame exclusivo contendo:
  - Nome do servidor
  - Campo vazio para inferência manual do sexo
- Esse DataFrame passou a ser a fonte oficial da variável inferida.

### Exportação para CSV
- O DataFrame auxiliar foi exportado para um arquivo CSV.
- O objetivo foi permitir uma inferência manual mais eficiente em ambiente visual (ex.: Excel ou LibreOffice).

### Inferência manual do sexo
- Cada nome foi analisado individualmente.
- O sexo foi inferido com base no primeiro nome do servidor.
- Foi adotada a seguinte convenção:
  - `M` para masculino
  - `F` para feminino

### Reimportação do arquivo tratado
- Após a inferência manual, o arquivo CSV foi reimportado para o ambiente de análise.
- A normalização dos nomes foi reaplicada para garantir consistência.

### Integração com o dataset principal
- A variável `sexo_inferido` foi incorporada ao dataset principal por meio de *merge*.
- A junção foi realizada utilizando o nome normalizado como chave.

### Validação final
- Foi realizada a verificação de valores nulos na variável `sexo_inferido`.
- Caso identificados, os registros foram corrigidos diretamente no CSV e o processo de importação e *merge* foi repetido.


In [29]:
nomes_unicos = (
    df_prepared["nome"]
    .dropna()
    .drop_duplicates()
    .sort_values()
    .reset_index(drop=True)
)

nomes_unicos

0      ACACIO DONIZETTI DE OLIVEIRA
1                 ADA ESTER ARCHILA
2           ADAO RODRIGUES DE JESUS
3                 ADELINI PELINGRIN
4                ADEMILTON MONTEIRO
                   ...             
975            WILSON RANOEL VERGNA
976               YARA MARIA ISABEL
977          YASMIN MICKAELA TAMINI
978           ZAIRA ANGELINA ROGADO
979                   ZILDA MESSINA
Name: nome, Length: 980, dtype: object

In [30]:
df_sexo_servidores = pd.DataFrame({
    "nome_servidor": nomes_unicos,
    "sexo_inferido": pd.NA
})

df_sexo_servidores

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,<NA>
1,ADA ESTER ARCHILA,<NA>
2,ADAO RODRIGUES DE JESUS,<NA>
3,ADELINI PELINGRIN,<NA>
4,ADEMILTON MONTEIRO,<NA>
...,...,...
975,WILSON RANOEL VERGNA,<NA>
976,YARA MARIA ISABEL,<NA>
977,YASMIN MICKAELA TAMINI,<NA>
978,ZAIRA ANGELINA ROGADO,<NA>


In [31]:
path_output = Path("../data/interim/sexo_servidores.csv")

df_sexo_servidores.to_csv(
    path_output,
    index=False,
    encoding="utf-8"
)

In [32]:
df_sexo_serv = pd.read_csv("../data/interim/inf_sexo_servidor.csv", sep=";")

df_sexo_serv.head()

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,M
1,ADA ESTER ARCHILA,F
2,ADAO RODRIGUES DE JESUS,M
3,ADELINI PELINGRIN,F
4,ADEMILTON MONTEIRO,M


In [33]:
def normalizar_nome(nome):
    return (
        nome
        .str.upper()
        .str.strip()
    )

df_prepared["nome_servidor_norm"] = normalizar_nome(df_prepared["nome"])
df_sexo_serv["nome_servidor_norm"] = normalizar_nome(df_sexo_serv["nome_servidor"])

In [34]:
df_prepared = df_prepared.merge(
    df_sexo_serv[["nome_servidor_norm", "sexo_inferido"]],
    on="nome_servidor_norm",
    how="left"
)

In [35]:
df_prepared["sexo_inferido"].value_counts(dropna=False)

sexo_inferido
F    14784
M     9213
Name: count, dtype: int64

In [36]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M


## Anonimização dos dados dos servidores

A partir deste ponto, os nomes dos servidores deixam de ser utilizados diretamente no processo de análise.

Embora os dados analisados sejam públicos, a anonimização é adotada como **boa prática em Ciência de Dados**, especialmente quando o objetivo é:

- Focar em padrões, distribuições e comportamentos agregados
- Evitar exposição desnecessária de informações pessoais
- Tornar o projeto reutilizável em outros contextos e datasets sensíveis

### Estratégia adotada

A anonimização foi realizada por meio da **geração de um identificador único irreversível** para cada servidor, utilizando uma função de hash criptográfico.

Características da abordagem:

- O mesmo nome sempre gera o mesmo identificador
- O processo é determinístico
- Não é possível recuperar o nome original a partir do identificador
- O identificador pode ser usado com segurança como chave em análises futuras

### Algoritmo utilizado

Foi utilizado o algoritmo **SHA-256**, disponível na biblioteca padrão do Python (`hashlib`), por ser amplamente adotado e seguro para este tipo de aplicação.

### Impacto na análise

Após a criação do identificador anonimizado:

- O nome do servidor deixa de ser necessário para análises estatísticas
- As análises passam a utilizar exclusivamente o identificador (`id_servidor`)
- Variáveis como sexo inferido, remuneração e período permanecem preservadas

Essa abordagem garante um equilíbrio entre **qualidade analítica**, **reprodutibilidade** e **responsabilidade no uso dos dados**.


In [41]:
def gerar_id_servidor(nome):
    if pd.isna(nome):
        return None
    return hashlib.sha256(nome.encode("utf-8")).hexdigest()

df_prepared["id_servidor"] = df_prepared["nome_servidor_norm"].apply(gerar_id_servidor)

In [42]:
df_prepared[["nome_servidor_norm", "id_servidor"]].head()

,nome_servidor_norm,id_servidor
0,ACACIO DONIZETTI DE OLIVEIRA,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...
1,ACACIO DONIZETTI DE OLIVEIRA,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...
2,ADA ESTER ARCHILA,348b58a2d1473088be45887aaa8b05de2944865139083c...
3,ADA ESTER ARCHILA,348b58a2d1473088be45887aaa8b05de2944865139083c...
4,ADAO RODRIGUES DE JESUS,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...


## Tratando a coluna cargo

In [44]:
df_prepared["cargo"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: cargo
Non-Null Count  Dtype 
--------------  ----- 
23997 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


Nesta coluna foi identificado 23997 linhas sem valores nulos

### Quantidade de cargos distintos

In [47]:
df_prepared["cargo"].nunique()

127

In [87]:
sorted(df_prepared["cargo"].unique())

['AGENTE ADMINISTRATIVO',
 'AGENTE COMUNITARIO DE SAUDE.',
 'AGENTE DE COMBATE AS ENDEMIAS',
 'AGENTE DE DESENVOLVIMENTO INFANTIL',
 'AGENTE DE SERVIÇOS DE SAÚDE',
 'AGENTE DE VIGILANCIA SANITARIA',
 'AJUDANTE DE ENCANADOR',
 'AJUDANTE DE PEDREIRO',
 'AJUDANTE DE SERVICOS DIVERSOS',
 'ASSESSOR DE GABINETE DE DIRETOR DE DEPARTAMENTO.c',
 'ASSESSOR DE GABINETE.c',
 'ASSESSOR DE IMPLEMENTAÇÃO DE POLÍTICAS PÚBLICAS.c',
 'ASSESSOR DE PLANEJAMENTO.c',
 'ASSISTENTE SOCIAL',
 'ATENDENTE SOCIAL',
 'AUXILIAR DE CRECHE',
 'AUXILIAR DE CUIDADOR DE CRIANCA - ABRIGO INSTITUCIONAL',
 'AUXILIAR DE DENTISTA PSF',
 'AUXILIAR DE ENFERMAGEM',
 'AUXILIAR DE FARMACIA',
 'AUXILIAR DE MANUTENCAO',
 'AUXILIAR DE MEDICOS DENTISTA',
 'AUXILIAR DE SERVICOS EXTERNOS',
 'BIBLIOTECARIA',
 'BORRACHEIRO',
 'CARPINTEIRO',
 'CHEFE DE GABINETE.c',
 'CIRURGIAO DENTISTA PSF',
 'CONSELHEIRO TUTELAR.',
 'CONTADOR',
 'CONTROLADOR INTERNO',
 'COVEIRO',
 'DENTISTA',
 'DIRETOR DO DEP. TURISMO, DESEN. ECO., CULTURA E ESPORTES.c',

- A listagem completa dos cargos foi inspecionada visualmente para verificação de consistência e padronização.
- Cargo é considerado comissionado quando o nome do cargo termina com “.c”.

## Criando categoria por cargo

- Normalizando a coluna cargo

In [74]:
def normalizar_texto(texto):
    if pd.isna(texto):
        return texto

    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ASCII", "ignore").decode("ASCII")

    texto = texto.upper()

    texto = texto.strip()
    texto = re.sub(r"\s+", " ", texto)


    return texto

df_prepared["cargo_norm"] = df_prepared["cargo"].apply(normalizar_texto)

In [75]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Efetivo,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,Efetivo,NaN,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Efetivo,NaN,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,Efetivo,NaN,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Efetivo,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO


- Mapeando os cargos

In [94]:
mapa_categoria_cargo = {
    # ADMINISTRATIVO / GESTAO
    "AGENTE ADMINISTRATIVO": "administrativo",
    "OFICIAL ADMINISTRATIVO": "administrativo",
    "CONTADOR": "administrativo",
    "CONTROLADOR INTERNO": "administrativo",
    "GESTOR DE PLANEJAMENTO": "administrativo",
    "FISCAL": "administrativo",

    # SAUDE
    "AGENTE COMUNITARIO DE SAUDE.": "saude",
    "AGENTE DE COMBATE AS ENDEMIAS": "saude",
    "AGENTE DE SERVICOS DE SAUDE": "saude",
    "AGENTE DE VIGILANCIA SANITARIA": "saude",
    "AUXILIAR DE DENTISTA PSF": "saude",
    "AUXILIAR DE ENFERMAGEM": "saude",
    "AUXILIAR DE FARMACIA": "saude",
    "AUXILIAR DE MEDICOS DENTISTA": "saude",
    "CIRURGIAO DENTISTA PSF": "saude",
    "DENTISTA": "saude",
    "ENFERMEIRO": "saude",
    "FARMACEUTICO": "saude",
    "FISIOTERAPEUTA": "saude",
    "FONOAUDIOLOGA": "saude",
    "MEDICO": "saude",
    "MEDICO ANESTESISTA": "saude",
    "MEDICO CARDIOLOGISTA": "saude",
    "MEDICO CIRURGIAO": "saude",
    "MEDICO CLINICO GERAL": "saude",
    "MEDICO DO TRABALHO": "saude",
    "MEDICO ENDOCRINOLOGISTA": "saude",
    "MEDICO GINECOLOGISTA/OBSTETRA": "saude",
    "MEDICO NEUROLOGISTA ADULTO": "saude",
    "MEDICO ORTOPEDISTA / TRAUMATOLOGISTA": "saude",
    "MEDICO PEDIATRA": "saude",
    "MEDICO PRONTO ATENDIMENTO": "saude",
    "MEDICO PSF": "saude",
    "MEDICO PSIQUIATRA ADULTO": "saude",
    "MEDICO UROLOGISTA": "saude",
    "NUTRICIONISTA": "saude",
    "TECNICO EM ENFERMAGEM": "saude",
    "TECNICO EM NUTRICAO": "saude",
    "VETERINARIO": "saude",

    # EDUCACAO
    "AGENTE DE DESENVOLVIMENTO INFANTIL": "educacao",
    "AUXILIAR DE CRECHE": "educacao",
    "INSPETOR DE ALUNOS": "educacao",
    "MERENDEIRA": "educacao",
    "MONITOR DE EDUCACAO FISICA": "educacao",
    "MESTRE DE MUSICA": "educacao",
    "PROFESSOR": "educacao",
    "PROFESSOR DE EDUCACAO FISICA": "educacao",
    "PROFESSOR DE EDUCACAO BASICA I - PEB I": "educacao",
    "PROFESSOR DE EDUCACAO BASICA I - PEB I - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - ARTE - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - CIENCIAS - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - EDUCACAO FISICA PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - GEOGRAFIA - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - INFORMATICA": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - INGLES - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - MATEMATICA - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - PORTUGUES - PD": "educacao",
    "PROFESSOR DE EDUCACAO INFANTIL - PEI - PD": "educacao",
    "PROFESSOR DE EDUCACAO INFANTIL -PEI": "educacao",
    "PROFESSOR DE ENSINO ESPECIAL": "educacao",
    "PROFESSOR DE SALA DE APOIO (PSA) EDUCACAO ESPECIAL - PD": "educacao",
    "PROFESSOR DE SALA DE APOIO (PSA-EDUC.ESPECIAL)": "educacao",
    "PROFESSOR EDUC.INFANTIL(CRECHE)-PEI-C": "educacao",
    "PROFESSOR EDUCACAO BASICA II - ARTE": "educacao",
    "PROFESSOR EDUCACAO BASICA II - EDUCACAO FISICA": "educacao",
    "PROFESSOR EDUCACAO BASICA II - GEOGRAFIA": "educacao",
    "PROFESSOR EDUCACAO BASICA II - LINGUA PORTUGUESA": "educacao",
    "PROFESSOR ENS FUND-CICLO I - II": "educacao",
    "PROFESSOR ENS FUND-CICLO III - IV": "educacao",
    "PROFESSOR SALA DE APOIO (PSA) EDUCACAO ESPECIAL": "educacao",

    # ASSISTENCIA SOCIAL
    "ASSISTENTE SOCIAL": "assistencia_social",
    "ATENDENTE SOCIAL": "assistencia_social",
    "ORIENTADOR SOCIAL": "assistencia_social",
    "CONSELHEIRO TUTELAR.": "assistencia_social",
    "AUXILIAR DE CUIDADOR DE CRIANCA - ABRIGO INSTITUCIONAL": "assistencia_social",
    "SUPERVISOR DE VISITAS": "assistencia_social",
    "PSICOLOGO": "assistencia_social",
    "PSICOLOGO INFANTIL": "assistencia_social",

    # OPERACIONAL / SERVICOS GERAIS
    "AJUDANTE DE ENCANADOR": "operacional",
    "AJUDANTE DE PEDREIRO": "operacional",
    "AJUDANTE DE SERVICOS DIVERSOS": "operacional",
    "AUXILIAR DE MANUTENCAO": "operacional",
    "AUXILIAR DE SERVICOS EXTERNOS": "operacional",
    "BORRACHEIRO": "operacional",
    "CARPINTEIRO": "operacional",
    "COVEIRO": "operacional",
    "ELETRICISTA": "operacional",
    "ENCANADOR": "operacional",
    "JARDINEIRO": "operacional",
    "LEITURISTA": "operacional",
    "LIXEIRO": "operacional",
    "MECANICO II": "operacional",
    "MOTORISTA": "operacional",
    "OPERADOR DA EBA": "operacional",
    "OPERADOR DE MAQUINA II": "operacional",
    "OPERADOR DE VACA MECANICA": "operacional",
    "PADEIRO": "operacional",
    "PEDREIRO": "operacional",
    "SERVENTE": "operacional",
    "VIGIA": "operacional",

    # TECNICO
    "TECNICO EM INFORMATICA": "tecnico",
    "TECNICO EM QUIMICA INDUSTRIAL (ETE)": "tecnico",
    "TECNICO SEGURANCA DO TRABALHO": "tecnico",

    # CULTURA / TURISMO
    "BIBLIOTECARIA": "cultura",
    "TURISMOLOGO": "cultura",

    # JURIDICO
    "PROCURADOR JURIDICO": "juridico",

    # POLITICO / ALTA GESTAO
    "PREFEITO": "politico",
    "VICE PREFEITO": "politico",

    # COMISSIONADOS / CARGOS DE CONFIANCA
    "ASSESSOR DE GABINETE DE DIRETOR DE DEPARTAMENTO.C": "comissionado",
    "ASSESSOR DE GABINETE.C": "comissionado",
    "ASSESSOR DE IMPLEMENTACAO DE POLITICAS PUBLICAS.C": "comissionado",
    "ASSESSOR DE PLANEJAMENTO.C": "comissionado",
    "CHEFE DE GABINETE.C": "comissionado",
    "GESTOR ADJUNTO DE ENSINO FUNDAMENTAL.C": "comissionado",
    "PROCURADOR GERAL DO MUNICIPIO.C": "comissionado",
    "DIRETOR DO DEP. TURISMO, DESEN. ECO., CULTURA E ESPORTES.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE ADMINISTRACAO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE AGRICULTURA E MEIO AMBIENTE.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE ASSISTENCIA SOCIAL.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE DESENVOLVIMENTO ECONOMICO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE DESENVOLVIMENTO URBANO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE EDUCACAO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE FINANCAS.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE GESTAO DE PESSOAS.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE OBRAS E ENGENHARIA.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE SAUDE.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE SERVICOS MUNICIPAIS.C": "comissionado",
}


In [95]:
df_prepared["categoria_cargo"] = df_prepared["cargo_norm"].map(mapa_categoria_cargo)

In [96]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


- Validando se algum cargo ficou sem catgoria

In [103]:
# Validando se algum cargo ficou sem categoria

df_prepared["categoria_cargo"].isna().sum()

np.int64(0)

In [98]:
(
    df_prepared
    .loc[df_prepared["categoria_cargo"].isna(), "cargo_norm"]
    .drop_duplicates()
    .sort_values()
)


Series([], Name: cargo_norm, dtype: object)

In [99]:
(
    df_prepared["categoria_cargo"]
    .isna()
    .mean()
    * 100
)


np.float64(0.0)

In [100]:
df_prepared["cargo_categorizado"] = df_prepared["categoria_cargo"].notna()

In [101]:
df_prepared["cargo_categorizado"].value_counts()

cargo_categorizado
True    23997
Name: count, dtype: int64

In [102]:
(
    df_prepared
    .loc[df_prepared["categoria_cargo"].isna(), "cargo_norm"]
    .drop_duplicates()
    .sort_values()
)


Series([], Name: cargo_norm, dtype: object)

In [116]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


- Conforme visto acima, todos os cargos foram mapeados

## Tratando coluna data_admissao

- Verificando nulos

In [120]:
df_prepared["data_admissao"].isna().sum()

np.int64(0)

In [121]:
df_prepared["data_admissao"].describe()

count          23997
unique           440
top       2025-02-03
freq            1066
Name: data_admissao, dtype: object

- Transformando o tipo da coluna para date

In [122]:
df_prepared["data_admissao"] = (
    pd.to_datetime(
        df_prepared["data_admissao"],
        errors="coerce",
        dayfirst=True
    )
    .dt.date
)

In [123]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


In [119]:
type(df_prepared['data_admissao'].dropna().iloc[0])

datetime.date

## Tratando coluna tipo_regime

In [124]:
df_prepared["tipo_regime"].describe

<bound method NDFrame.describe of 0        REGIME PROPRIO
1        REGIME PROPRIO
2        REGIME PROPRIO
3        REGIME PROPRIO
4        REGIME PROPRIO
              ...      
23992    REGIME PROPRIO
23993    REGIME PROPRIO
23994    REGIME PROPRIO
23995    REGIME PROPRIO
23996    REGIME PROPRIO
Name: tipo_regime, Length: 23997, dtype: object>

In [125]:
sorted(df_prepared["tipo_regime"].unique())

['REGIME GERAL  ', 'REGIME PROPRIO']

No contexto de servidores públicos municipais/estaduais:

Regime Geral (geralmente “RGPS” – Regime Geral de Previdência Social):
Normalmente se refere a servidores efetivos, que ingressaram via concurso público.
Contribuem para o sistema de previdência geral (INSS).
São considerados ocupantes de cargos estáveis, com direitos típicos de servidores concursados.


Regime Próprio (geralmente “RPPS” – Regime Próprio de Previdência Social):
Pode incluir servidores comissionados (cargos de confiança) ou efetivos, dependendo do município/estado.
Para cargos comissionados, geralmente não há estabilidade e podem ser exonerados a qualquer momento.
Contribuem para o regime próprio de previdência do ente federativo (municipal ou estadual).

Sendo assim, será mantido o formato da coluna e não passará por nenhuma transformação

### Tratamento da coluna `desconto`

A coluna `desconto` representa os valores descontados do pagamento do servidor.

Observações iniciais:
- Quando o pagamento é referente ao **vale alimentação**, não há desconto, portanto a coluna apresenta `NaN`.
- Quando o pagamento é referente ao **salário (folha mensal)**, o valor do desconto vem no formato brasileiro, por exemplo: `1.500,00`.

#### Objetivos do tratamento:
1. Zerar a coluna `desconto` para os pagamentos do tipo **vale alimentação**.
2. Converter os valores de salário para o formato **float**, pronto para análises financeiras.
3. Garantir que não haja valores nulos (`NaN`) na coluna após o tratamento.

#### Passo a passo:

1. **Identificar o tipo de pagamento**
   - A coluna `tipo_pagamento` indica se o registro é `folha_mensal` ou `vale_alimentacao`.

2. **Função para conversão de string pt-br para float**

In [129]:
def str_para_float(valor):
    if pd.isna(valor):
        return 0.0

    return float(str(valor).replace(".", "").replace(",", "."))

df_prepared["descontos"] = np.where(
    df_prepared["tipo_pagamento"] == "vale_alimentacao", 0.0,
    df_prepared["descontos"].apply(str_para_float)
)

In [130]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,"3.603,75",NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,"5.432,23",NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


In [132]:
df_prepared["descontos"].describe

<bound method NDFrame.describe of 0           0.00
1        1584.02
2           0.00
3        1330.84
4           0.00
          ...   
23992     535.56
23993       0.00
23994     323.08
23995    1947.27
23996       0.00
Name: descontos, Length: 23997, dtype: float64>

In [133]:
df_prepared["descontos"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: descontos
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


### Tratamento da coluna `liquido`

A coluna `liquido` representa o valor líquido pago ao servidor em cada pagamento.

Observações iniciais:
- Os valores estão no **formato brasileiro**, por exemplo `1.500,00`.
- Para análises financeiras, precisamos do formato **float**, por exemplo `1500.00`.
- Não há necessidade de diferenciar por tipo de pagamento; todos os registros devem ser convertidos.

#### Objetivos do tratamento:
1. Converter os valores para **float**.
2. Garantir que não haja valores `NaN`.
3. Padronizar para análises financeiras e cálculos futuros.

In [134]:
df_prepared["liquido"].describe

<bound method NDFrame.describe of 0        1.000,00
1        3.603,75
2        1.000,00
3        5.432,23
4        1.000,00
           ...   
23992    2.920,63
23993    1.000,00
23994    2.442,78
23995    3.576,00
23996    1.000,00
Name: liquido, Length: 23997, dtype: object>

In [135]:
df_prepared["liquido"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: liquido
Non-Null Count  Dtype 
--------------  ----- 
23997 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [137]:
def str_para_float(valor):
    if pd.isna(valor):
        return 0.0
    # Substitui ponto como separador de milhar e vírgula como decimal
    return float(str(valor).replace('.', '').replace(',', '.'))

df_prepared["liquido"] = df_prepared["liquido"].apply(str_para_float)

In [138]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True
